In [66]:
import pandas as pd
import json

pd.options.display.max_columns = 100
pd.reset_option("display.max_colwidth")

In [63]:
with open('respuesta-fravega.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [64]:
# 2️⃣ Extraer los productos dentro del JSON
productos = data["data"]["items"]["results"]

In [67]:
df = pd.DataFrame(productos)
cols = ['id', 'title', 'katalogCategoryId', 'brand', 'skus','slug']
df = df[cols]
df.head()

,id,title,katalogCategoryId,brand,skus,slug
0,68ad8b1dac9baed38210bbf4,Asus Rog Flow X13 Amd Ryzen 9 16gb ssd1tb gtx ...,5a301c031400008a0049181b,"{'id': '5a301bdc140000630049072b', 'name': 'As...","{'results': [{'code': '22591597', 'categorizat...",asus-rog-flow-x13-amd-ryzen-9-16gb-ssd1tb-gtx-...
1,68c9a32f10aa48b1124182b2,Asus Vivobook Rosa Core 5 16GB RAM 1TB SSD 14 ...,5a301c031400008a0049181b,"{'id': '5a301bdc140000630049072b', 'name': 'As...","{'results': [{'code': '990249853', 'categoriza...",asus-vivobook-rosa-core-5-16gb-ram-1tb-ssd-14-...
2,68c9a34a4057014b7fc264ef,Asus Vivobook Rosa Core 5 16GB RAM 512GB SSD 1...,5a301c031400008a0049181b,"{'id': '5a301bdc140000630049072b', 'name': 'As...","{'results': [{'code': '990249850', 'categoriza...",asus-vivobook-rosa-core-5-16gb-ram-512gb-ssd-1...
3,68c9a37e52c064a49f4b536a,Asus Vivobook Rosa Core 5 2TB SSD 40GB RAM 14 ...,5a301c031400008a0049181b,"{'id': '5a301bdc140000630049072b', 'name': 'As...","{'results': [{'code': '990249854', 'categoriza...",asus-vivobook-rosa-core-5-2tb-ssd-40gb-ram-14-...
4,67f97e4108044aa2211fb90a,"Notebook ASUS Vivobook Go 15 15,6” Intel Core ...",5a301c031400008a0049181b,"{'id': '5a301bdc140000630049072b', 'name': 'As...","{'results': [{'code': '364693', 'categorizatio...",notebook-asus-vivobook-go-15-15-6”-intel-core-...


In [69]:
df = pd.json_normalize(df.to_dict('records'), sep='_')
df = df.drop(columns=['brand___typename','skus___typename'], axis=1)

In [71]:

# ---------- helpers ----------
def ensure_list(v):
    return v if isinstance(v, list) else []

def first_list(v):
    return v[0] if isinstance(v, list) and v else None

def get_category_names_and_slugs(categ):
    """
    'categ' suele ser una lista de rutas (cada ruta es una lista de dicts con name/slug).
    Tomamos la PRIMERA ruta y devolvemos nombres y slugs.
    """
    if not isinstance(categ, list) or not categ:
        return [], []
    ruta = categ[0]
    if not isinstance(ruta, list):
        return [], []
    names = [x.get("name") for x in ruta if isinstance(x, dict)]
    slugs = [x.get("slug") for x in ruta if isinstance(x, dict)]
    return names, slugs

# ---------- 1) Exploto SKUs: una fila por SKU ----------
tmp = df.copy()
tmp["skus_results"] = tmp["skus_results"].apply(ensure_list)
tmp = tmp.explode("skus_results", ignore_index=True)

# Normalizo el dict del SKU en columnas prefijadas
sku_cols = pd.json_normalize(tmp["skus_results"]).add_prefix("sku_")
tmp = tmp.drop(columns=["skus_results"]).join(sku_cols)

# ---------- 2) Me quedo con el pricing preferido (fravega-ecommerce si existe) ----------
# Exploto pricing a filas
tmp["sku_pricing"] = tmp["sku_pricing"].apply(ensure_list)
tmp = tmp.explode("sku_pricing", ignore_index=True)

pricing_cols = pd.json_normalize(tmp["sku_pricing"]).add_prefix("pricing_")
tmp = tmp.drop(columns=["sku_pricing"]).join(pricing_cols)

# Para cada sku_code, quedarme con 'fravega-ecommerce' si existe, sino la primera disponible
# Ordeno para que fravega-ecommerce quede primero
tmp["_pref_order"] = (tmp["pricing_channel"] != "fravega-ecommerce").astype(int)
tmp = tmp.sort_values(["sku_code", "_pref_order"]).drop_duplicates(subset=["sku_code"], keep="first").drop(columns=["_pref_order"])

# ---------- 3) Categorías desde sku_categorization ----------
# sku_categorization suele ser list[list[dict{name, slug}]]
names_slugs = tmp["sku_categorization"].apply(get_category_names_and_slugs)
tmp["category_names"] = names_slugs.apply(lambda x: x[0])
tmp["category_slugs"] = names_slugs.apply(lambda x: x[1])

tmp["category_path"] = tmp["category_names"].apply(lambda ns: " > ".join(ns) if ns else None)
tmp["category_lvl_1"] = tmp["category_names"].apply(lambda ns: ns[0] if isinstance(ns, list) and len(ns) > 0 else None)
tmp["category_lvl_2"] = tmp["category_names"].apply(lambda ns: ns[1] if isinstance(ns, list) and len(ns) > 1 else None)

# ---------- 4) Selección final: ids, categorías, list/offer price ----------
salida = tmp[[
    "id",                 # id del producto
    "sku_code",           # id del SKU
    "category_lvl_1",
    "category_lvl_2",
    "category_path",
    "pricing_listPrice",  # precio de lista
    "pricing_salePrice"   # precio de oferta
]].rename(columns={
    "id": "product_id",
    "sku_code": "sku_id",
    "pricing_listPrice": "list_price",
    "pricing_salePrice": "sale_price"
})

In [77]:
def ensure_list(v):
    return v if isinstance(v, list) else []

def first_list(v):
    return v[0] if isinstance(v, list) and v else None

def get_category_names(categ):
    """
    'categ' suele ser una lista de rutas (cada ruta es una lista de dicts con name/slug).
    Retorna una lista con todos los nombres de categorías.
    """
    if not isinstance(categ, list) or not categ:
        return []
    ruta = categ[0]
    if not isinstance(ruta, list):
        return []
    return [x.get("name") for x in ruta if isinstance(x, dict)]

# ---------- 1) Exploto SKUs: una fila por SKU ----------
tmp = df.copy()
tmp["skus_results"] = tmp["skus_results"].apply(ensure_list)
tmp = tmp.explode("skus_results", ignore_index=True)

# Normalizo el dict del SKU en columnas
sku_cols = pd.json_normalize(tmp["skus_results"]).add_prefix("sku_")
tmp = tmp.drop(columns=["skus_results"]).join(sku_cols)

# ---------- 2) Exploto pricing ----------
tmp["sku_pricing"] = tmp["sku_pricing"].apply(ensure_list)
tmp = tmp.explode("sku_pricing", ignore_index=True)
pricing_cols = pd.json_normalize(tmp["sku_pricing"]).add_prefix("pricing_")
tmp = tmp.drop(columns=["sku_pricing"]).join(pricing_cols)

# ---------- 3) Priorizar canal 'fravega-ecommerce' ----------
tmp["_pref_order"] = (tmp["pricing_channel"] != "fravega-ecommerce").astype(int)
tmp = tmp.sort_values(["sku_code", "_pref_order"]).drop_duplicates(subset=["sku_code"], keep="first").drop(columns=["_pref_order"])

# ---------- 4) Obtener categorías como lista ----------
tmp["categories"] = tmp["sku_categorization"].apply(get_category_names)

# ---------- 5) Selección final ----------
df_skus = tmp[[
    "id",                 # ID del producto
    "sku_code",           # Código del SKU
    "categories",         # Lista de categorías
    "pricing_listPrice",  # Precio de lista
    "pricing_salePrice"   # Precio de oferta
]].rename(columns={
    "sku_code": "sku_id",
    "pricing_listPrice": "list_price",
    "pricing_salePrice": "sale_price"
})

In [ ]:
df = df.merge(df_skus, on='id').drop('skus_results', axis=1)

,id,title,katalogCategoryId,slug,brand_id,brand_name,sku_id,categories,list_price,sale_price
0,68ad8b1dac9baed38210bbf4,Asus Rog Flow X13 Amd Ryzen 9 16gb ssd1tb gtx ...,5a301c031400008a0049181b,asus-rog-flow-x13-amd-ryzen-9-16gb-ssd1tb-gtx-...,5a301bdc140000630049072b,Asus,22591597,"[Notebooks, Informática]",8999999.00,5979000.00
1,68c9a32f10aa48b1124182b2,Asus Vivobook Rosa Core 5 16GB RAM 1TB SSD 14 ...,5a301c031400008a0049181b,asus-vivobook-rosa-core-5-16gb-ram-1tb-ssd-14-...,5a301bdc140000630049072b,Asus,990249853,"[Notebooks, Informática]",2207414.00,1709466.00
2,68c9a34a4057014b7fc264ef,Asus Vivobook Rosa Core 5 16GB RAM 512GB SSD 1...,5a301c031400008a0049181b,asus-vivobook-rosa-core-5-16gb-ram-512gb-ssd-1...,5a301bdc140000630049072b,Asus,990249850,"[Notebooks, Informática]",1857524.00,1356366.00
3,68c9a37e52c064a49f4b536a,Asus Vivobook Rosa Core 5 2TB SSD 40GB RAM 14 ...,5a301c031400008a0049181b,asus-vivobook-rosa-core-5-2tb-ssd-40gb-ram-14-...,5a301bdc140000630049072b,Asus,990249854,"[Notebooks, Informática]",2642324.00,2148366.00
4,67f97e4108044aa2211fb90a,"Notebook ASUS Vivobook Go 15 15,6” Intel Core ...",5a301c031400008a0049181b,notebook-asus-vivobook-go-15-15-6”-intel-core-...,5a301bdc140000630049072b,Asus,364693,"[Notebooks, Informática]",1199999.00,699999.00
5,67f97e42bf733aac4d3cc7cb,"Notebook ASUS Vivobook Go 15 15,6” AMD Ryzen 5...",5a301c031400008a0049181b,notebook-asus-vivobook-go-15-15-6”-amd-ryzen-5...,5a301bdc140000630049072b,Asus,364730,"[Notebooks, Informática]",1299999.00,1099999.00
6,67f97e42c7979ac7dd271606,"Notebook Asus Vivobook 15 15,6” AMD Ryzen 7 8G...",5a301c031400008a0049181b,notebook-asus-vivobook-15-15-6”-amd-ryzen-7-8g...,5a301bdc140000630049072b,Asus,364745,"[Notebooks, Informática]",1849999.00,1299999.00
7,685ac886050cd74564401161,Notebook ASUS Vivobook 15 Intel® Core™ i7 16GB...,5a301c031400008a0049181b,notebook-asus-vivobook-15-intel®-core™-i7-16gb...,5a301bdc140000630049072b,Asus,364657,"[Notebooks, Informática]",1799999.00,1499999.00
8,67f97e4176c4fb1f831bf6fc,"Notebook ASUS TUF Gaming F15 15,6"" NVIDIA GeFo...",5a301c031400008a0049181b,notebook-asus-tuf-gaming-f15-15-6-nvidia-gefor...,5a301bdc140000630049072b,Asus,364595,"[Notebooks, Informática]",3299999.00,2799999.00
9,685ac8869fe01779e6fa50a0,Notebook ASUS Vivobook S16 Intel® Core™ i7 16G...,5a301c031400008a0049181b,notebook-asus-vivobook-s16-intel®-core™-i7-16g...,5a301bdc140000630049072b,Asus,364662,"[Notebooks, Informática]",2199999.00,1949999.00
